Start by loading the PETLION package

In [2]:
using PETLION

Loading the parameters for an LCO cell

In [3]:
@time p = Params(LCO;
    N_p = 10, # discretizations in the cathode
    N_s = 10, # discretizations in the separator
    N_n = 10, # discretizations in the anode
    N_r_p = 10, # discretizations in the solid cathode particles
    N_r_n = 10, # discretizations in the solid anode particles
    temperature = false, # temperature enabled or disabled
    jacobian    = :symbolic # :symbolic or :AD
    );

 42.472472 seconds (6.74 M allocations: 411.641 MiB, 0.28% gc time, 7.38% compilation time)


# 1. Input options
Once the model is loaded, discretizations and numerical options (temperature, aging, Fickian diffusion, etc.) cannot be modifed.

What **can** be modified is in `p.opts` and `p.bounds`.

`p.opts`:
- `p.opts.SOC` -  Initial state of charge for a new simulation between 0 and 1
- `p.opts.outputs` -  Saving model states is expensive. What states do you want to keep? See the output of `model` below for more info. Must be a tuple
- `p.opts.abstol` -  Absolute tolerance of DAE solver
- `p.opts.reltol` -  Relative tolerance of DAE solver
- `p.opts.maxiters` -  Maximum iterations for the DAE solver
- `p.opts.check_bounds` -  Flag to check the bounds during simulation (SOC max/min, V max/min, etc.)
- `p.opts.reinit` -  Get a new initial guess for DAE initialization
- `p.opts.verbose` -  Show some outputs during simulation runtime
- `p.opts.interp_final` -  Interpolate the final results to match the exact simulation end point
- `p.opts.tstops` -  Times when the DAE solver explicitly stops
- `p.opts.tdiscon` -  For input functions, times when there is a known discontinuity. Unknown discontinuities are handled automatically but less efficiently
- `p.opts.interp_bc` -  `:interpolate` or `:extrapolate` when interpolating the model

`p.bounds` (`NaN` deactivates the bound):
- `p.bounds.V_max` - Maximum permitted voltage [V]
- `p.bounds.V_min` - Minimum permitted voltage [V]
- `p.bounds.SOC_max` - Maximum permitted SOC [-]
- `p.bounds.SOC_min` - Minimum permitted SOC [-]
- `p.bounds.T_max` - Maximum permitted temperature [K]
- `p.bounds.c_s_n_max` - Maximum permitted solid surface concentration in the anode [mol/m³]
- `p.bounds.I_max` - Maximum permitted current [C-rate]
- `p.bounds.I_min` - Minimum permitted current [C-rate]
- `p.bounds.η_plating_min` - Minimum permitted plating overpotential at the separator-anode interface [V]
- `p.bounds.c_e_min` - Minimum permitted electrolyte concentration [mol/m³]

# 2. Model output options
As an example, let's run a CC-CV simulation. *Everything* that may be modified in `p.opts` can also be modified as an input to `run_model`. Modifcations in `run_model`, however, are *only* for that particular run.

In [5]:
model = run_model(p, I=2, SOC=0, V_max=4.1)
run_model!(model, p, V=:hold)

PETLION model
  --------
  Runs:    I → V
  Time:    2444.59 s
  Voltage: 4.1 V
  Current: 0.1921C
  Power:   23.0214 W
  SOC:     1.0
  Exit:    Above maximum SOC limit

The `model` outputs are:
- `I`- Current [C-rate]
- `t`- Time [s]
- `V`- Voltage [V]
- `P`- Power [W]
- `SOC`- State-of-charge [-]
- `c_e`- Electrolyte concentrations [mol/m³]
- `c_s_avg`- Volume-averaged solid concentrations [mol/m³]
- `T`- Temperature (if temperature is enabled) [K]
- `film`- Film thickness (if aging == :SEI) [m]
- `Q`- Variable for polynomial (if solid_diffusion is :polynomial)
- `j`- Ionic flux [mol/(m²⋅s)]
- `j_s`- Side reaction flux (if aging == :SEI) [mol/(m²⋅s)]
- `Φ_e`- Electrolyte potential [V]
- `Φ_s`- Solid potential [V]
- `Y`- All algebraic states
- `YP`- All differential states
- `results`- Info about the model output

In [7]:
model.V

123-element Vector{Float64}:
 2.8634951045878543
 2.8651702533686687
 2.8665685699664576
 2.869281089163381
 2.8744019417044675
 2.8792244132313263
 2.8837821119130584
 2.8923203176169383
 2.900065444313087
 2.913597080358639
 2.923964214800758
 2.9330345532855047
 2.941064435076053
 ⋮
 4.1
 4.1
 4.1
 4.1
 4.1000000000000005
 4.1
 4.1
 4.099999999999999
 4.100000000000001
 4.099999999999999
 4.1000000000000005
 4.1000000000000005

For example, since `:c_e` was not selected as a default output of the model, we can't don't have results for it

In [9]:
model.c_e

VectorOfArray{Float64,2}:
Vector{Float64}[]

We can change that by doing `p.opts.outputs = (:t, :V, :c_e)` or `p.opts.outputs = (:all)`

In [137]:
p.opts.outputs = (:t, :V, :c_e,)
model = run_model(p, I=2, SOC=0, V_max=4.1)
run_model!(model, p, V=:hold)
model.c_e

VectorOfArray{Float64,2}:
122-element Vector{Vector{Float64}}:
 [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0  …  1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
 [1000.0246810515931, 1000.0255005909374, 1000.027200738911, 1000.0299017077414, 1000.0338159252412, 1000.0393000315828, 1000.0469554914413, 1000.0578298774842, 1000.0738433624605, 1000.0982176658913  …  999.9471382773901, 999.954526237071, 999.9605427717266, 999.9652205057847, 999.9688624041816, 999.9716723192477, 999.9737883432888, 999.9753038644774, 999.9762803748988, 999.9767552218999]
 [1000.0457507650931, 1000.0472657655997, 1000.0504083287896, 1000.0553999827179, 1000.062631865899, 1000.0727598545764, 1000.0868880708701, 1000.1069344537989, 1000.1363988185016, 1000.1807072159274  …  999.9029275852266, 999.9161029789898, 999.9271110841456, 999.9356863409843, 999.9423711399703, 999.9475335342705, 999.9514236668683, 999.9542111162893, 999.9560077413081, 999.

To **only** get the results of one particular run, we can index the model

In [139]:
model[1] # CC section

PETLION model
  --------
  Run:     I
  Time:    1388.68 s
  Voltage: 4.1 V
  Current: 0.1921C
  Power:   23.023 W
  SOC:     1.0
  Exit:    Above maximum voltage limit

In [140]:
model[end] # CV section

PETLION model
  --------
  Run:     V
  Time:    2445.64 s
  Voltage: 4.1 V
  Current: 0.1921C
  Power:   23.023 W
  SOC:     1.0
  Exit:    Above maximum SOC limit

We can also post-interpolate the model

In [130]:
t_interp = 0:1:model.t[end]
model_interp = model(t_interp)

PETLION model
  --------
  Runs:    I → V
  Time:    2444.0 s
  Voltage: 4.1 V
  Current: 0.1921C
  Power:   23.0214 W
  SOC:     1.0
  Exit:    Above maximum SOC limit